In [6]:
import pandas as pd
import openpyxl

df = pd.read_excel('data/reactiva.xlsx', sheet_name = 'TRANSFERENCIAS 2020')
df.head(0)

,ID,CODIGO PAIS,CODIGO ENTIDAD,UBIGEO,SNIP,CUI,REGION,PROVINCIA,DISTRITO,PROYECTO,...,AMBITO,UNIDAD EJECUTORA,TOTAL EMPLEOS,POBLACION BENEFICIARIA,TIPOLOGIA,TIPO MONEDA,MONTO DE INVERSIÓN,TIPO MONEDA.1,MONTO DE TRANSFERENCIA 2020,ESTADO


In [16]:
def limpiar_nombres_columnas(df):
    # Convertir nombres de columna a minúsculas
    df.columns = df.columns.str.lower()
    # Eliminar espacios en los nombres de columna
    df.columns = df.columns.str.replace(' ', '_')
    # Eliminar tildes en los nombres de columna
    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    return df

df_1 = limpiar_nombres_columnas(df)
df_1.head(1)

,id,codigo_pais,codigo_entidad,ubigeo,snip,cui,region,provincia,distrito,proyecto,...,ambito,unidad_ejecutora,total_empleos,poblacion_beneficiaria,tipologia,tipo_moneda,monto_de_inversion,tipo_moneda.1,monto_de_transferencia_2020,estado
0,1,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,...,URBANO,MUNICIPALIDAD DISTRITAL DE QUICHUAY,121,1529,Pista y Vereda,PEN,4105693.0,PEN,4105693,En Ejecución


In [18]:
df_2 = df_1.drop(['id', 'tipo_moneda.1'], axis= 1)
df_2.head(1)

,codigo_pais,codigo_entidad,ubigeo,snip,cui,region,provincia,distrito,proyecto,dispositivo_legal,ambito,unidad_ejecutora,total_empleos,poblacion_beneficiaria,tipologia,tipo_moneda,monto_de_inversion,monto_de_transferencia_2020,estado
0,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,"DU N° 070-2020,",URBANO,MUNICIPALIDAD DISTRITAL DE QUICHUAY,121,1529,Pista y Vereda,PEN,4105693.0,4105693,En Ejecución


In [29]:
df_3 = df_2
df_3['dispositivo_legal'] = df_3['dispositivo_legal'].replace({'0m':''}, regex=True)
df_3.head(1)

,codigo_pais,codigo_entidad,ubigeo,snip,cui,region,provincia,distrito,proyecto,dispositivo_legal,ambito,unidad_ejecutora,total_empleos,poblacion_beneficiaria,tipologia,tipo_moneda,monto_de_inversion,monto_de_transferencia_2020,estado
0,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,DU N° 070-2020,URBANO,MUNICIPALIDAD DISTRITAL DE QUICHUAY,121,1529,Pista y Vereda,PEN,4105693.0,4105693,En Ejecución


In [38]:
!pip install requests
import requests
from datetime import date


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def obtener_tipo_cambio_sunat(fecha):
    try:
        url = f"https://api.apis.net.pe/v1/tipo-cambio-sunat?fecha={fecha}"
        response = requests.get(url)
        response.raise_for_status()
        return response.json()['compra']
    except requests.RequestException as e:
        print("Error al obtener el tipo de cambio:", e)
        return None

fecha_actual = date.today().strftime('%Y-%m-%d')
tipo_cambio_usd = obtener_tipo_cambio_sunat(fecha_actual)

df_4 = df_3
df_4['monto_inversion_dol'] = (df_4['monto_de_inversion'] / tipo_cambio_usd).round(2)
df_4['monto_transferencia2020_dol'] = (df_4['monto_de_transferencia_2020'] / tipo_cambio_usd).round(2)

In [54]:
df_4 = df_4.drop('monto_dolares', axis=1)

In [55]:
df_4.head(3)

,codigo_pais,codigo_entidad,ubigeo,snip,cui,region,provincia,distrito,proyecto,dispositivo_legal,...,unidad_ejecutora,total_empleos,poblacion_beneficiaria,tipologia,tipo_moneda,monto_de_inversion,monto_de_transferencia_2020,estado,monto_inversion_dol,monto_transferencia2020_dol
0,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,DU N° 070-2020,...,MUNICIPALIDAD DISTRITAL DE QUICHUAY,121,1529,Pista y Vereda,PEN,4105693.00,4105693,En Ejecución,1079593.22,1079593.22
1,PE,11476,90612,213528,2215425,HUANCAVELICA,HUAYTARA,SAN ISIDRO,MEJORAMIENTO VIAL DEL CERCADO DE LA LOCALIDAD ...,DU N° 070-2020,...,MUNICIPALIDAD DISTRITAL DE SAN ISIDRO - HUIRPA...,63,648,Pista y Vereda,PEN,2120069.47,413572,En Ejecución,557472.91,108748.88
2,PE,11476,60501,321607,2276250,CAJAMARCA,CONTUMAZA,CONTUMAZA,CREACION DEL SERVICIO DE TRANSITABILIDAD VEHIC...,DU N° 070-2020,...,MUNICIPALIDAD PROVINCIAL DE CONTUMAZA,92,645,Pista y Vereda,PEN,3157699.42,1219289,Concluido,830318.02,320612.41


In [63]:
df_5 = df_4
df_5['estado'] = df_5['estado'].replace('En Ejecución', 'Ejecución')
df_5['estado'] = df_5['estado'].replace('Convenio y/o Contrato Resuelto', 'Resuelto')

In [64]:
df_5.estado.unique()

array(['Ejecución', 'Concluido', 'Actos Previos', 'Resuelto'],
      dtype=object)

In [71]:
def puntuar(estado):
    valor = estado
    if (valor == 'Resuelto'):
        puntuacion = 0
    elif valor == 'Actos Previos' :
        puntuacion = 1
    elif valor == 'Ejecución' :
        puntuacion = 2
    elif valor == 'Concluido':
        puntuacion = 3
    else:
        puntuacion = None
    return puntuacion
    
df_final = df_5
df_final['puntuación'] = df_final['estado'].apply(puntuar)
df_final.head(2)

,codigo_pais,codigo_entidad,ubigeo,snip,cui,region,provincia,distrito,proyecto,dispositivo_legal,...,total_empleos,poblacion_beneficiaria,tipologia,tipo_moneda,monto_de_inversion,monto_de_transferencia_2020,estado,monto_inversion_dol,monto_transferencia2020_dol,puntuación
0,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,DU N° 070-2020,...,121,1529,Pista y Vereda,PEN,4105693.00,4105693,Ejecución,1079593.22,1079593.22,2
1,PE,11476,90612,213528,2215425,HUANCAVELICA,HUAYTARA,SAN ISIDRO,MEJORAMIENTO VIAL DEL CERCADO DE LA LOCALIDAD ...,DU N° 070-2020,...,63,648,Pista y Vereda,PEN,2120069.47,413572,Ejecución,557472.91,108748.88,2


In [72]:
df_final.columns

Index(['codigo_pais', 'codigo_entidad', 'ubigeo', 'snip', 'cui', 'region',
       'provincia', 'distrito', 'proyecto', 'dispositivo_legal', 'ambito',
       'unidad_ejecutora', 'total_empleos', 'poblacion_beneficiaria',
       'tipologia', 'tipo_moneda', 'monto_de_inversion',
       'monto_de_transferencia_2020', 'estado', 'monto_inversion_dol',
       'monto_transferencia2020_dol', 'puntuación'],
      dtype='object')

In [73]:
import sqlite3

# Conectar a la base de datos (se creará si no existe)
conexion = sqlite3.connect('ubicaciones.db')

# Extraer datos únicos de las columnas 'ubigeo', 'Region', 'Provincia' y 'Distrito'
datos_ubigeo = df_final[['ubigeo', 'region', 'provincia', 'distrito']].drop_duplicates()

# Crear una tabla en la base de datos SQLite y almacenar los datos
datos_ubigeo.to_sql('ubigeo', conexion, if_exists='replace', index=False)

# Confirmar y cerrar la conexión
conexion.commit()
conexion.close()

print("Tabla de ubigeos almacenada en la base de datos.")

Tabla de ubigeos almacenada en la base de datos.


In [78]:
# Tipo de obra 'Urbano' y estado 1, 2 o 3
condicion1 = (df_final['tipologia'] == 'Equipamiento Urbano') & (df_final['puntuación'] >= 1) & (df_final['puntuación'] <= 3)
df_filtrado = df_final[condicion1]

# Lista de regiones unicas
Lista_reg_unicas = df_filtrado['region'].unique()

for region in Lista_reg_unicas:
    # Filtrar los datos por región
    condicion2 = df_filtrado['region'] == region
    df_region = df_filtrado[condicion2]
    # Verificar si hay datos para la región actual
    if not df_region.empty:
        
        df_region_ordenado = df_region.sort_values(by='monto_de_inversion', ascending=False)
        
        top_5_obras = df_region_ordenado.head(5)
        
        # Guardar el resultado en un archivo Excel
        nombre_archivo = f"t5_inversion_{region}.xlsx"
        top_5_obras.to_excel(nombre_archivo, index=False)
        print(f"Archivo '{nombre_archivo}' generado correctamente.")
    else:
        print(f"No hay datos para la región '{region}'. No se generará el reporte.")

Archivo 't5_inversion_LIMA.xlsx' generado correctamente.
Archivo 't5_inversion_APURIMAC.xlsx' generado correctamente.
Archivo 't5_inversion_PASCO.xlsx' generado correctamente.
Archivo 't5_inversion_AYACUCHO.xlsx' generado correctamente.
Archivo 't5_inversion_CAJAMARCA.xlsx' generado correctamente.
Archivo 't5_inversion_ICA.xlsx' generado correctamente.
Archivo 't5_inversion_SAN MARTIN.xlsx' generado correctamente.
Archivo 't5_inversion_JUNIN.xlsx' generado correctamente.
Archivo 't5_inversion_TACNA.xlsx' generado correctamente.
Archivo 't5_inversion_PUNO.xlsx' generado correctamente.
Archivo 't5_inversion_PIURA.xlsx' generado correctamente.
Archivo 't5_inversion_HUANUCO.xlsx' generado correctamente.
Archivo 't5_inversion_AMAZONAS.xlsx' generado correctamente.
Archivo 't5_inversion_CUSCO.xlsx' generado correctamente.
Archivo 't5_inversion_LA LIBERTAD.xlsx' generado correctamente.
Archivo 't5_inversion_HUANCAVELICA.xlsx' generado correctamente.
Archivo 't5_inversion_LORETO.xlsx' generado

In [79]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [83]:
import os 



smtp_server = 'smtp.gmail.com'  # Cambia esto al servidor SMTP que estés utilizando
smtp_port = 587  # Cambia esto al puerto adecuado
sender_email = 'alelplm17@gmail.com'
#sender_password = open('token.txt').read().strip()
sender_password = 'wwsu nuzq wwjp zhnm'

# Detalles del correo electrónico
receiver_email = 'p.m.alejandro.2015@gmail.com' # 'p.m.alejandro.2015@gmail.com'  'alejandro.pariona2@unmsm.edu.pe' 'alepharim@gmail.com'
subject = 'Re-envio Reporte Reactiva por regiones'
body = 'Adjunto lo solicitado y los archivos py, ipynb y excel'

# Crear el objeto MIMEMultipart
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))


# Adjuntar archivo
file_paths = ['Problema1.ipynb', 'Problema2.ipynb', 'procesamiento.py', 't5_inversion_AMAZONAS.xlsx']  # Cambia la ruta al archivo que quieras adjuntar
for file_path in file_paths:
    with open(file_path, 'rb') as file:
        attachment = MIMEApplication(file.read(), _subtype="csv")
        attachment.add_header('Content-Disposition', 'attachment', filename=file_path)
        msg.attach(attachment)
    
# Iniciar la conexión con el servidor SMTP
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()  # Iniciar el modo seguro
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

print('Correo enviado exitosamente')

Correo enviado exitosamente
